# Weir: Water Flowing Over a Low Dam
***
https://en.wikipedia.org/wiki/Weir

This is an example of a 2D fluid flow.
***
## Setup the Environment
***

In [1]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../')))

In [2]:
import numpy

MatPlotLib is used for creating custom visualizations

In [3]:
import matplotlib.pyplot as plt

In [4]:
import spatialpy

***
## Create the Boundary Conditions for the System
***
For custom boundary conditions it is necessary to subclass the `BoundaryCondition` class that implements the `__init__` and `expression` methods. The `expression` method must return a `C++` executable code block in string format.

For this example we create two custom boundry conditions:
- `Teleport`:   Simulates the constant flow of fluid particles (without the need to create and destroy them) by teleporting them to the begining of the system.
- `HardSurface`:  Simulates the solid ground, preventing particles from passing through.

In [5]:
class Teleport(spatialpy.BoundaryCondition):
    def __init__(self):
        pass
    
    def expression(self):
        return """
        if(me->x[0] > system->xhi){
            me->x[0] = system->xlo + 0.55;
        }
        if(me->x[0] < system->xlo ){
            me->x[0] = system->xlo + 0.55;
        }
        if(me->x[1] > system->yhi){
            me->v[1]= 0.0;
            me->x[1]= system->yhi - 0.01;
        }
        me->x[2] = 0;
        me->rho = 2.5;
        """

In [6]:
class HardSurface(spatialpy.BoundaryCondition):
    def __init__(self, type_ids):
        self.type_ids = type_ids
    
    def expression(self):
        exp = "if("
        exp += f"me->x[1] < 0.4"
        for type_id in self.type_ids:
            exp += f" && me->type != {type_id}"
        exp += "){"
        exp += "me->x[1] = 0.5;"
        exp += "me->v[1] = 0.0;"
        exp += "}"
        return exp

***
## Create Geometries for the Domain
***
For custom geometries it is necessary to subclass the `Geometry` class that implements the `__init__` and `inside` methods. The `inside` method must return boolean value.

For this example we create three custom geometries: `StreamBed`, `Dam`, and `Water`

In [7]:
class StreamBed(spatialpy.Geometry):
    def __init__(self, y_lim):
        self.y_lim = y_lim
        
    def inside(self, point, on_boundary):
        if point[1] == self.y_lim:
            return True
        return False

In [8]:
class Dam(spatialpy.Geometry):
    def __init__(self, x_lim, y_lim):
        self.x_lim = x_lim
        self.y_lim = y_lim
        
    def inside(self, point, on_boundary):
        if point[0] >= self.x_lim[0] and point[0] <= self.x_lim[1] and \
                    point[1] > self.y_lim[0] and point[1] < self.y_lim[1]:
            return True
        return False

In [9]:
class Water(spatialpy.Geometry):
    def __init__(self, x_lim, y_lim):
        self.x_lim = x_lim
        self.y_lim = y_lim
        
    def inside(self, point, on_boundary):
        if point[1] > self.y_lim[0] and point[1] < self.y_lim[1] and \
                    point[0] > self.x_lim[0] and point[0] < self.x_lim[1]:
            return True
        return False

***
## Creating a Fluid Dynamics Model using SpatialPy
***

In [10]:
def create_weir(model_name="weir", parameter_values=None):
    # Initialize Model
    model = spatialpy.Model(model_name)

    # Define Domain Type IDs as constants of the Model
    model.STREAM_BED = StreamBed.__name__
    model.DAM = Dam.__name__
    model.WATER = Water.__name__

    # Set Domain bounds
    xmax = 35.1
    ymax = 10.1
    delta_x = 0.2

    # Set mass per particle
    water_mass = 1.0
    solid_mass = 1.0

    """
    Create an empty domain
    - numpoints: Total number of spatial domain points.
    - xlim: Range of domain along x-axis.
    - ylim: Range of domain along y-axis.
    - zlim: Range of domain along z-axis.
    - rho0: Background density for the system.
    - c0: Speed of sound for the system.
    - P0: Background pressure for the system.
    - gravity: Acceleration of gravity for the system.
    """
    domain = spatialpy.Domain(
        numpoints=0, xlim=(0, xmax), ylim=(0, ymax), zlim=(0, 0), gravity=[0, -1, 0]
    )
    # Fill our domain with particles using Geometries
    """
    Fill a geometric shape with particles.

    - geometry_ivar: an instance of a 'spatialpy.Geometry' subclass.  The 'inside()' method
                     of this object will be used to add a single point particles.
    - deltax: Distance between particles on the x-axis.
    - deltay: Distance between particles on the y-axis (defaults to deltax).
    - deltaz: Distance between particles on the z-axis (defaults to deltax).
    - xmin: Minimum x value of the bounding box (defaults to Domain.xlim[0]).
    - xmax: Maximum x value of the bounding box (defaults to Domain.xlim[1]).
    - ymin: Minimum y value of the bounding box (defaults to Domain.ylim[0]).
    - ymax: Maximum y value of the bounding box (defaults to Domain.ylim[1]).
    - zmin: Minimum z value of the bounding box (defaults to Domain.zlim[0]).
    - zmax: Maximum z value of the bounding box (defaults to Domain.zlim[1]).
    - type_id: Particle type ID of particle to be created.
    - vol: Default volume of particle to be added.
    - mass: Default mass of particle to be added.
    - nu: Default viscosity of particle to be created.
    - c: Default artificial speed of sound of particle to be created.
    - rho: Default density of particle to be created
    - fixed: True if particle is spatially fixed, else False.
    """
    # Create the stream bed
    np_bed = domain.fill_with_particles(
        geometry_ivar=StreamBed(0),
        deltax=delta_x,
        ymax=0,
        type_id=model.STREAM_BED,
        mass=solid_mass,
        vol=1.0,
        nu=1.0,
        fixed=True
    )
    # Create the dam
    np_dam = domain.fill_with_particles(
        geometry_ivar=Dam((10, 15), (0, 5)),
        deltax=delta_x,
        xmin=10,
        xmax=15,
        ymax=5,
        type_id=model.DAM,
        mass=solid_mass,
        vol=1.0,
        nu=1.0,
        fixed=True
    )
    # Create the water
    np_water = domain.fill_with_particles(
        geometry_ivar=Water((1, 8), (1, 8)),
        deltax=delta_x * 2,
        xmin=1,
        xmax=8,
        ymin=1,
        ymax=8,
        type_id=model.WATER,
        mass=water_mass,
        vol=1.0,
        nu=1.0,
        fixed=False
    )
    # Set Model Domain
    model.add_domain(domain)

    # Define Boundary Conditions
    teleport = Teleport()
    hard_surface = HardSurface(
        (model.domain.get_type_def(model.STREAM_BED), model.domain.get_type_def(model.DAM))
    )
        
    # Add the Boundary Conditions to the Model
    model.add_boundary_condition([teleport, hard_surface])

    # Setting staticDomain to False allows particles to move within the system.
    model.staticDomain = False

    # Define Timespan
    tspan = spatialpy.TimeSpan.linspace(t=50, num_points=101, timestep_size=1e-3)
    
    # Use NumPy to set the timespan of the Model.
    model.timespan(tspan)
    return model

### Instantiate your Model

In [11]:
model = create_weir()

AttributeError: 'list' object has no attribute 'model'

***
## Run the  Simulation
***

In [ ]:
results = model.run()

***
## Visualization
***
Plot the results of the simulation.

For fluid dynamics problems visualizing particle properies can be valuable, lets plot the `Type IDs` of the particles at the start of the simulation using MatPlotLib. For this example lets generate custom plots for timepoint `[0, 1, 10, 99]` using `MatPlotLib` 

In [ ]:
def process_data(points, raw_data):
    # Sort data by Type ID
    data = {}
    for i, val in enumerate(raw_data['type']):
        name = model.domain.typeNameMapping[val][5:]

        if name in data:
            data[name].append(points[i])
        else:
            data[name] = [points[i]]
    return data

In [ ]:
def mpl_plot_property(t_ndx=0):
    """
    Read the data for 'step_num'. Returns a tuple containing a numpy.ndarray 
    of point coordinates [0] along with a dictionary of property and species data [1]

    - step_num: The index in the results timespan.
    """
    points, raw_data = results.read_step(t_ndx)
    # Process raw data for plotting
    data = process_data(points=points, raw_data=raw_data)
    
    # Set the colors for each type_id
    colors = ["brown", "grey", "blue"]
    
    xlim = results.model.domain.xlim
    ylim = results.model.domain.ylim
    aspect_ratio = ylim[1] / xlim[1]
    width = 8.25
    
    fig, ax = plt.subplots(figsize=(width, width * aspect_ratio))
    ax.set_xlim(xlim[0] - 1, xlim[1] + 1)
    ax.set_ylim(ylim[0] - 1, ylim[1] + 1)
    for i, (name, points) in enumerate(data.items()):
        points = numpy.array(points)
        ax.scatter(points[:, 0], points[:, 1], c=colors[i], label=name)
        ax.grid(linestyle='', linewidth=1)
        ax.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1, fontsize=12)
    plt.plot()
    return

In [ ]:
mpl_plot_property(0)

In [ ]:
mpl_plot_property(1)

In [ ]:
mpl_plot_property(10)

In [ ]:
mpl_plot_property(99)

Just like before that plot is somewhat boring, so lets plot the `Type IDs` over time using Plotly anumation.

In [ ]:
results.plot_property(
    # Set the name of the property.
    property_name='type',
    
    # Set to True to use Plotly animation
    animated=True,
    
    # Set the transition and frame durations
    t_duration=10,
    f_duration=10,
)

Other properties that are available for vizualization are:
- Velocity: `v`
- Density: `rho`
- Mass: `mass`
- Viscocity: `nu`
- Boundary Volume Fraction: `bvf_phi` (non-standard SDPD: https://www.sciencedirect.com/science/article/abs/pii/S0955799721000916)

Lets plot the velocity in the `X` and `Y` directions.

In [ ]:
results.plot_property(
    # Set the name of the property.
    property_name='v',
    
    # Set the velocity direction (X: 0, Y: 1, Z: 0)
    p_ndx=0,
    
    # Set to True to use Plotly animation
    animated=True,
    
    # Set the transition and frame durations
    t_duration=10,
    f_duration=10
)

In [ ]:
results.plot_property(
    # Set the name of the property.
    property_name='v',
    
    # Set the velocity direction (X: 0, Y: 1, Z: 0)
    p_ndx=1,
    
    # Set to True to use Plotly animation
    animated=True,
    
    # Set the transition and frame durations
    t_duration=10,
    f_duration=10
)

A more complete list of `results.plot_property()` arguments can be found in the [documentation](https://stochss.github.io/SpatialPy/docs/build/html/classes/spatialpy.core.html#spatialpy.core.result.Result.plot_property).